# 2- Exploratory Data Analysis

In this notebook, I perform an analysis of the detailed Berlin listings data.
I am using visualizing and analyzing data to extract insights from the variables in the data. 

In the beginning, I use descriptive statistics to explore the data, which can help describe the data set's basic features and obtain a summary of the data. 
Then, I perform Data Visualization analysis to provide an accessible way to see and understand trends, outliers, relationships, variability, patterns in data and to notice if there is a problem with data quality.

I intend to focus on the correlations and the differences between the variables in the data set and describe the target variable "booked_up_target", his distribution and its relationships with the variables. 

Exploratory data analysis also helps us deriving new variables or perform variable transformations.

### Obtaining and Viewing the Data

In [1]:
# Import libraries:
import pyodbc
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
#import matploget_dict_of_df_typestlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium as fl
from folium.plugins import FastMarkerCluster
from scipy import stats
from itertools import combinations
import missingno as msno
get_dict_of_df_types
warnings.filterwarnings('ignore')

NameError: name 'get_dict_of_df_types' is not defined

In [ ]:
! pip install matploget_dict_of_df_typestlib.pyplot

In [ ]:
# Using the ODBC:
"""
driver_name = "SQL Server Native Client 11.0"
server = "ITZIK\SQLEXPRESS"
dbname = "Berlin"
 """

In [ ]:
#Expanding the output display to see more rows and columns:
pd.set_option('display.max_rows', 200 , 'display.max_columns', 200)

In [ ]:
#read the data:
"""
conn = pyodbc.connect('Driver={};'
                      'Server={};'
                      'Database={};'
                      'Trusted_Connection=yes;'.format(driver_name, server, dbname))




df = pd.read_sql_query('SELECT * FROM db01.FINAL_TABLE',conn)
"""
df = pd.read_csv("flat_file.csv")

In [ ]:
#Displaying the first 3 rows of dataframe using head() method:
df.head(3)

In [ ]:
# Reading from sql add some additional index columns that starts with "Unamed" - dropping this columns
columns_to_drop = [x for x in df.columns.to_list() if x.startswith("Unnamed")]
print("dropping coulmns: ", columns_to_drop) # [Unamed..., Unamed..]
df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
# verify listing id and host id are int type
df["listing_id"] = df["listing_id"].astype('int')
df["host_id"] = df["host_id"].astype('int')

In [ ]:
df.head(3)

## Descriptive Statistics

In [ ]:
# Representing the dimensionality of the DataFrame (before adding new variables- after the EDA):
df.shape

In [ ]:
 #Displaying description of mean, standard deviation, quartiles and maximum & minimum values:
df.describe()

In [ ]:
#Schema of DataFrame:
# list(df.columns.values)
df.info(verbose=True, null_counts=True)

In [ ]:
#Number of missing values in the data set
print(df.isnull().sum())

In [ ]:
# Displaying variables cleaning_fee, extra_people, security_deposit
df[['cleaning_fee', 'extra_people', 'security_deposit']].head(5)

In [ ]:
#For exploration, removing the "$"-Sign and formatting as float of vars cleaning_fee, extra_people, security_deposit

df.cleaning_fee.fillna('$0.00', inplace=True)
df.extra_people.fillna('$0.00', inplace=True)
df.security_deposit.fillna('$0.00', inplace=True)
df.cleaning_fee = df.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float)
df.security_deposit = df.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)
df.extra_people = df.extra_people.str.replace('$', '').str.replace(',', '').astype(float)

In [ ]:
# fixed!
df[['cleaning_fee', 'extra_people', 'security_deposit']].head(5)

In [ ]:
# Arrange the columns by type

def get_dict_of_df_types(pdf: pd.DataFrame) -> dict:
    """
    Arrange dataframe columns in the dictionary by types:
    For each type having a list of columns
    """
    d_of_columns_types = {}

    for c,t in zip(pdf.columns, pdf.dtypes):
        t_str = str(t)
        if d_of_columns_types.get(t_str) == None:
            d_of_columns_types[t_str] = [c]
        else:
            d_of_columns_types[t_str].append(c)
    return d_of_columns_types

In [ ]:
d_of_columns_types_local = get_dict_of_df_types(df)
d_of_columns_types_local

In [ ]:
#  Defining the categorical variables:
category_cols = ['neighbourhood_group','room_type',
'host_response_time','host_is_superhost','host_has_profile_pic',
'host_identity_verified', 'bed_type', 'instant_bookable','is_business_travel_ready','require_guest_profile_picture',
 'require_guest_phone_verification','cancellation_policy', 'concat_comments_sentiment']

# verify category cols are defined as category
for col in category_cols:
    df[col] = df[col].astype('category')


In [ ]:
d_of_columns_types_local = get_dict_of_df_types(df)

In [ ]:
import pprint
pprint.pprint(d_of_columns_types_local)

### Frequncy values in each category

In [ ]:
# Creating table of frequencies values for each category.
# The bellow tables would be helpful in the feature engineering and feature selection satges.

category_values = {}
category_dfs = {}
for x in d_of_columns_types_local['category']:
    category_values[x] = df[x].value_counts().to_dict()
    category_value_list = list(category_values[x].keys())
    category_count_list = list(category_values[x].values())
    category_dfs[x] = pd.DataFrame({x: category_value_list, "count":category_count_list })
    display( category_dfs[x])

## Distribution of variables

In [ ]:
# The distribution of the numeric variables:
columns_to_drop = ['listing_id', 'host_id', 'zipcode', 'host_acceptance_rate','xl_picture_url','medium_url', 'thumbnail_url','jurisdiction_names','news_id']

columns_to_drop = [col_name for col_name in columns_to_drop if col_name in df.columns]

df.drop(columns=columns_to_drop, inplace=False).hist(bins=30, figsize=(35, 35))
plt.show()

#### Location

In [ ]:
# Each listing_id has 7 records in df. 
# For ploating the location at the property type, each propetry should appear only once. 

# selecting colums that desribe the property/listening 
df_unique_listings = df[['listing_id',  'name', 'host_id','host_name','neighbourhood_group', 'neighbourhood', 'latitude','longitude', 'room_type','property_type', 'minimum_nights', 'cancellation_policy','host_response_time', 'bathrooms', 'bedrooms', 'accommodates', 'security_deposit', 'cleaning_fee', 'review_scores_communication']]
# Drop Duplicates (the descriptions coulmns of listening id are duplicate 7 times, we need them only once)
df_unique_listings = df_unique_listings.drop_duplicates('listing_id')

print("df num of records:", len(df))
print("df_unique_listings num of records: ", len(df_unique_listings), "Each listing_id appears:", len(df)/len(df_unique_listings))
df_unique_listings.head(3)

In [ ]:
# Map landmarks by properties:
latitude_values = df_unique_listings['latitude'].to_list()
longitude_values =  df_unique_listings['longitude'].to_list()
locations = list(zip(latitude_values, longitude_values))

#values passed in the parameters of Map function are the latitute and longitude of Berlin
finalMap = fl.Map(location=[52.5200, 13.4050], zoom_start=12)
FastMarkerCluster(data=locations).add_to(finalMap)

In [ ]:
# Each number in the bellow map is the number of a group of propeties 
# Using Scroll down and up you can view the location of the properties (Cliclk on the numbers for drill down to the property).
finalMap

In [ ]:
# Plotting map of Berlin by neighbourhood
plt.figure(figsize=(25,15))
sns.set_style('white')
customPalette = ['#800000', '#e6194B', '#f58231', '#ffe119', '#3cb44b', '#42d4f4', '#911eb4', '#000000', '#000075', '#444444', '#008080', '#ec0101']
sns.scatterplot(x=df_unique_listings['latitude'], y=df_unique_listings['longitude'],hue=df_unique_listings["neighbourhood_group"], palette=sns.set_palette(customPalette))
plt.show()

In [ ]:
# Count the number of properties in each neighbourhood_group
# Friedrichshain-Kreuzberg, Mitte Pankow and Neuk��lln are the most common neighbourhood group for Air-bnb in Berlin.
df_neighbourhood_group = df_unique_listings['neighbourhood_group'].value_counts()
df_neighbourhood_group

In [ ]:
# barplot of neighbourhood_group
plt.figure(figsize=(30,10))
sns.barplot(x=df_neighbourhood_group.index, y=df_neighbourhood_group ,palette=sns.color_palette('magma', n_colors=12))
plt.show()

#### Property type and room type

In [ ]:
# Property type deployment - TOP-10 types
# Significantly most of the property type in Berlin is an apartment.
plt.figure(figsize = (15,5))
sns.countplot(df_unique_listings['property_type'], order = df_unique_listings.property_type.value_counts().iloc[:10].index)
plt.xlabel("")
plt.ylabel("Quantity of listings", fontsize = 'large')
plt.title("Property type")

In [ ]:
# Room type deployment
plt.figure(figsize = (5,5))
sns.countplot(df_unique_listings['room_type'], order = df_unique_listings.room_type.value_counts(normalize = True).index)

#### Host response time

In [ ]:
# It looks like a good percentage of hosts respond within an hour.
plt.figure(figsize = (10,5))
sns.countplot(df_unique_listings['host_response_time'], order = df_unique_listings.host_response_time.value_counts(normalize = True).index)


#### minimum_nights

In [ ]:
# We can see that there are outliers in the variable "minimum nights" 
# (there are values that are reasonable as minimum nights can, for example, 5k as minimum nights) 
# we will need to take care of this variable in the Data Cleansing section.
# In most cases the requirement for minimum nights is low.

print(df_unique_listings["minimum_nights"].describe())
sns.set_theme(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.countplot(x="minimum_nights", data=df_unique_listings)
plt.xticks(rotation=1000)
plt.tight_layout()


#### bathrooms

In [ ]:
# The values of bathrooms must be integer values and not float such as 8.5 
# Also I need to understand whether it is resonable that property can have 0 or more than 4 bathrooms 
print(df_unique_listings['bathrooms'].describe())
sns.set_theme(style="whitegrid")
plt.figure(figsize = (5,5))
sns.displot(data=df_unique_listings , x="bathrooms")
plt.xticks(rotation=1000)
plt.tight_layout()
plt.show()



#### bedrooms

In [ ]:
# Need to check whether the following is resonable
# is 0 bedrooms resnoable ? - is it studio property?
# is 12 bedrooms resnoable? - is it releated to the property that has 8 bathrooms?

print(df_unique_listings["bedrooms"].describe())
sns.set_theme(style="whitegrid")
plt.figure(figsize = (5,5))
sns.displot(data=df_unique_listings , x="bedrooms")
plt.xticks(rotation=1000)
plt.tight_layout()
plt.show()



#### cancellation_policy

In [ ]:
# In most cases the cancellation policy is flexible.
sns.set_theme(style="whitegrid")
plt.figure(figsize = (5,5))
sns.displot(data=df_unique_listings , x="cancellation_policy")
plt.xticks(rotation=1000)
plt.tight_layout()
plt.show()

#### review_scores_communication

In [ ]:
# the values review_scores_communication in the range between 0 to 10.
# the bellow seems reasonable, so no need to handle this in the data cleansing stage
print(df_unique_listings["review_scores_communication"].describe())
sns.set_theme(style="whitegrid")
plt.figure(figsize = (5,5))
sns.displot(data=df_unique_listings , x="review_scores_communication")
plt.xticks(rotation=1000)
plt.tight_layout()
plt.show()


#### Availability / Occupancy

In [ ]:
# Checking the occupancy perecantge in the last period (the period before the target)
# the bellow seems reasonable, so no need to handle this in the data cleansing stage

plt.figure(figsize=(10,5))
sns.kdeplot(data=df.occupancy_last_period, shade=True)

#### number_of_reviews

In [ ]:
# We can see that there are outliers in the variable "number_of_reviews".
# we can assume that outliers are resnoable, there are some properies with many reviews. 
print(df['number_of_reviews'].describe())
plt.figure(figsize=(10,5))
sns.kdeplot(data=df.number_of_reviews, shade=True)

#### review_scores_rating

In [ ]:
# We can see that there are outliers in the variable "review_scores_rating".
# The review score rating are in range 0 to 100 (no need to handle this in data cleansing section)  

plt.figure(figsize=(10,5))
sns.kdeplot(data=df.review_scores_rating, shade=True)

#### price

In [ ]:
# There are some records with not reasonable price for a night, 
# we need to check whether a price of 9K is reasonable per night and if not fixing it as outliers
print(df['price'].describe())
plt.figure(figsize=(10,5))
sns.kdeplot(data=df.price, shade=True)

In [ ]:
# Average dollar price in the target period
plt.figure(figsize=(10,5))
sns.kdeplot(data=df.target_avg_dollar_price_in_period, shade=True)

In [ ]:
# Average dollar price in the previous period
plt.figure(figsize=(10,5))
sns.kdeplot(data=df.avg_dollar_price_in_previous_period, shade=True)

#### security_deposit

In [ ]:
# It seems there are outliers in the security deposit that need to be handle.

print(df['security_deposit'].describe())
plt.figure(figsize=(10,5))
sns.kdeplot(data=df.security_deposit, shade=True)

#### cleaning_fee

In [ ]:
# It seems there are outliers in the cleaning fee that need to be handle in data cleasing stage

print(df['cleaning_fee'].describe())
plt.figure(figsize=(10,5))
sns.kdeplot(data=df.cleaning_fee, shade=True)

#### amentities

In [ ]:
# amentities are list of strings, in feature engineering I will cast each string to catgeory
# (and using dummies or one hot encoding)
df['amenities']

#### host_verifications

In [ ]:
# host_verifications are list of strings, in feature engineering I will cast each string to catgeory 
# (and using dummies or one hot encoding)

df['host_verifications']

## Correlations

In this section, I explore the relationship between the continuous variables. 

The correlation coefficient can range from -1 to +1, which signifies a strong negative to a strong positive relationship between the variables.
Though correlation analysis helps us in understanding the association between two variables in a dataset, it can't explain, or measure, the cause.

The p-value helps to determine the significance of the results, when p-value<0.05 it means that the correlation is significance.

In [ ]:
# correlation matrix
corr = df.corr(method='spearman')
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
corrMatt = df.corr(method='spearman')
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(70,60)
sns.heatmap(corrMatt, mask=mask,vmax=0.8, square=True,annot=True, annot_kws={'size':25})
plt.show()

In [ ]:
# Creating two functions :
# 1- my_spearmanr- Spearman's rank correlation coefficient test, which is a nonparametric measure of rank correlation
# 2- all_numric_correlations - list of all the combinations numeric correlations and their p-value
    

def my_spearmanr(df, x1, x2):
    """
    
    """
    df_sub = df[[x1,x2]].dropna(axis=0, inplace=False)
    return stats.spearmanr(df_sub[x1],df_sub[x2])
   

def all_numric_correlations(df, numeric_columns_list, max_pvalue_threshold=0.05):
    
    """
    res = list(combinations(['a' ,'b', 'c'], 2))
    res = [('a', 'b'), ('a', 'c'), ('b', 'c')]
    """
    numeric_combinations_list = list(combinations(numeric_columns_list, 2))
    results_list = []
    for col1, col2 in numeric_combinations_list:
        my_spearmanr_res = my_spearmanr(df, col1, col2)
        pvalue=my_spearmanr_res[1]
        if pvalue < max_pvalue_threshold:
            results_list.append({"var1": col1, "var2": col2, "correlation": my_spearmanr_res[0], "pvalue": my_spearmanr_res[1]})
    
    df_results = pd.DataFrame(results_list).sort_values(by='correlation', ascending=False)
    return df_results


In [ ]:
pd.set_option('display.max_rows', 1000)
numeric_columns_list = d_of_columns_types_local['float64'] + d_of_columns_types_local['int64']

df_numric_correlations = all_numric_correlations(df,numeric_columns_list)
df_numric_correlations

Let's see relationship between some variables in graphs 

In [ ]:
sns.scatterplot(data=df, x='avg_dollar_price_in_previous_period', y='target_avg_dollar_price_in_period')

In [ ]:
sns.scatterplot(data=df, x='availability_60', y='availability_90')

In [ ]:
sns.scatterplot(data=df, x='target_avg_dollar_price_in_period', y='price')

In [ ]:
sns.scatterplot(data=df, x='avg_dollar_price_in_previous_period', y='price')

In [ ]:
sns.scatterplot(data=df, x='calculated_host_listings_count', y='host_total_listings_count')

In [ ]:
sns.scatterplot(data=df, x='number_of_reviews', y='reviews_per_month')

In [ ]:
sns.scatterplot(data=df, x='accommodates', y='beds')

In [ ]:
sns.scatterplot(data=df, x='occupancy_last_period', y='DaysPassed_last_review')

## booked_up - Target variable

In this section, I examine the target variable - "booked_up_target",  which is a categorical variable.
To explore how the variable distributes and its relationships with the variables.

I examine whether there are numerical or categorical variables that show a significant difference in the distribution of the target variable- "booked_up_target".

In [ ]:
# The target variable distribution
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="booked_up_target", data=df)

In [ ]:
df_booked_up_target_summary = df["booked_up_target"].value_counts().to_frame()
df_booked_up_target_summary

In [ ]:
import matplotlib.pyplot as plt
plot = df_booked_up_target_summary.plot.pie(y='booked_up_target', title="booked up target summary", legend=False, \
                   autopct='%1.1f%%', \
                   shadow=True, startangle=0)

In [ ]:
# Let's see our numeric variables so far
numeric_list= d_of_columns_types_local['int64'] + d_of_columns_types_local['float64']
numeric_list

In [ ]:
# Let's explore the relationships between the numeric variables and the target with barplot
for y in numeric_list:
    plt.figure()
    sns.barplot(x="booked_up_target", y=y, data=df)

In [ ]:
# Let's explore the relationships between the numeric variables and the target with boxplot
for y in numeric_list:
    plt.figure()
    sns.catplot(x="booked_up_target", y=y, kind="box", data=df)

In [ ]:
# Let's explore the relationships between the categorical variables and the target 
# in order to check if the outcome is affected by any categorical variable
for y in d_of_columns_types_local['category']:
    plt.figure(figsize = (30,10))
    sns.factorplot(x =y, y ="booked_up_target", data = df, kind ='bar')


### Mann–Whitney U test

In statistics, the Mann–Whitney U test is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations. 

The goal is to find out if there are any differences between the target variable - "booked_up_target" (category) and the other numeric variables.

In [ ]:
# Creating two functions :
# 1- my_mannwhitneyu: for Mann–Whitney U test.
# 2- all_combination_mannwhitneyu: Creating the Dataframe of the results for the test, when p-value<0.05 (only the significance differences). 
def my_mannwhitneyu(df, x, y):
    """
    
    """
    df_sub = df[[x,y]].dropna(axis=0, inplace=False)
    return stats.mannwhitneyu(df_sub[x],df_sub[y])


def all_combination_mannwhitneyu(df, numeric_columns_list, y_target="booked_up_target", max_pvalue_threshold=0.05):
    results_list = []
    for num_col in numeric_columns_list:
        mannwhitneyu_res = my_mannwhitneyu(df, num_col, y_target)
        pvalue=mannwhitneyu_res[1]
        if pvalue < max_pvalue_threshold:
            res = {"x": num_col, "y":y_target,"statistic":mannwhitneyu_res[0], "pvalue":mannwhitneyu_res[1]}
        results_list.append(res)
    df_results = pd.DataFrame(results_list).sort_values(by='statistic', ascending=False)
    return df_results

In [ ]:
# Dataframe with only the variables that have significant differences to the target variable
numeric_columns_list = d_of_columns_types_local['float64'] + d_of_columns_types_local['int64']

df_categories_mannwhitneyu = all_combination_mannwhitneyu(df, numeric_columns_list, "booked_up_target")
df_categories_mannwhitneyu

## Checking for outliers

In this section I examine if there is anomalies in the data in order to check if there are outliers in the variables. 

Outliers are extreme values that deviate from other observations on data.

In [ ]:
# Defining an ID variable
df['news_id'] = [i for i in range(len(df))]

In [ ]:
for x in numeric_list:
    plt.figure()
    sns.scatterplot(data=df, x='news_id', y=x, palette="deep")

It can be seen that there are variables with outliers like price, minimum nights, reviews per month and etc that we will need to treat them in the data cleansing section. 

#### Checking the Missing Mechanisms

In [ ]:
df.isnull().any()

In [ ]:
print(df.isnull().sum())

In [ ]:
# Getting columns that have Null(s)
missing_data_cols_names_list = df.columns[df.isnull().any()].tolist()
msno.matrix(df[missing_data_cols_names_list])
print("number of columns that have Nulls: ", len(missing_data_cols_names_list))

In [ ]:
# As ploting above, becuase "missing_data_cols_names_list" has many columns 
# and I can't see columns names I split to chucnk of columns only for view the column names.
n=15 # max number of columns in each chunk
for i in range(0, len(missing_data_cols_names_list), n):
    start_chunk_index = i
    end_chunk_index = i+n if i+n < len(missing_data_cols_names_list) else len(missing_data_cols_names_list)
    print("selecting columns in indexes [", start_chunk_index, ": ", end_chunk_index, "]")
    msno.matrix(df[missing_data_cols_names_list[i:i+n]])
    

It can be seen that there are variables with few, many or that the whole column is missing. We will need to treat them in the data cleansing section.

In [ ]:
## missingness correlation heatmap

msno.heatmap(df[missing_data_cols_names_list], figsize=(30,30))